# Ablation Study Results

Axes
- model (gemma-3-4b-it, gemma-3-27b-it, gpt-5)
- manual vs judge
- experiment (base, subreddit, summary, liked posts list, fine tuned, soft prompt (100), soft prompt(500))
- dataset (circlejerk, jokes + puns, gaming, animals, sports, etc.)

### gemma-3-4b-it
|                   | Self Defined | Summary | Like History | Fine Tune | Soft Prompt (100) | Soft Prompt (500) |
|-------------------|-----------|---------|--------------|-----------|-------------------|-------------------|
| Circlejerk        |           |         |              |           |                   |                   |
| Jokes             |           |         |              |           |                   |                   |
| Gaming            |           |         |              |           |                   |                   |
| Animals           |           |         |              |           |                   |                   |
| Personal          |           |         |              |           |                   |                   |
| Personal + Gaming |           |         |              |           |                   |                   |

### gemma-3-27b-it
|                   | Self Defined | Summary | Like History | Fine Tune | Soft Prompt (100) | Soft Prompt (500) |
|-------------------|-----------|---------|--------------|-----------|-------------------|-------------------|
| Circlejerk        |           |         |              |           |                   |                   |
| Jokes             |           |         |              |           |                   |                   |
| Gaming            |           |         |              |           |                   |                   |
| Animals           |           |         |              |           |                   |                   |
| Personal          |           |         |              |           |                   |                   |
| Personal + Gaming |           |         |              |           |                   |                   |

### gpt-5
|                   | Self Defined | Summary | Like History | Fine Tune | Soft Prompt (100) | Soft Prompt (500) |
|-------------------|-----------|---------|--------------|-----------|-------------------|-------------------|
| Circlejerk        |           |         |              |           |                   |                   |
| Jokes             |           |         |              |           |                   |                   |
| Gaming            |           |         |              |           |                   |                   |
| Animals           |           |         |              |           |                   |                   |
| Personal          |           |         |              |           |                   |                   |
| Personal + Gaming |           |         |              |           |                   |                   |


In [ ]:
MODELS = [
    "gemma-3-4b-it",
    "gemma-3-27b-it",
    "gpt-5"
]

DATASETS = [
    "circlejerk",
    "jokes + puns",
    "gaming",
    "animals",
    "personal",
    "personal + gaming"
]

EXPERIMENTS = [
    "self defined",
    "summary",
    "like history",
    "fine tune",
    "soft prompt (100)",
    "soft prompt (500)"
]

## Dataset Creation

In [ ]:
# hand pick by scrolling through frontend
# add mechanism to save post to json file whenever liked by user 
# repeat 500 times for each category 

## Dataset Loading

In [ ]:
# write a generator to lazily load dataset from json file based on dataset argument

## Soft Prompt Training

In [ ]:
# iterate through models, datasets, and dataset sizes to create soft prompt adapters for each 
# 2 * 6 * 2 = 24 adapters
# take note of training time, adapter size

## Fine Tuning

In [ ]:
# iterate through models, datasets 
# 2 * 6 = 12 fine tuned models
# take note of training time, model size

## Generate posts

In [ ]:
# 100 posts per cell 
# look into inference parallelism
# iterate through models, experiments, datasets

## Judge LLM